<h1>Импорты</h1>

In [1821]:
import pandas as pd
import numpy as np
import pyodbc
import random
from sklearn.preprocessing import OrdinalEncoder
from datetime import datetime

db_conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-JSR6TV0G;'
                      'Database=Pro_t_londonSQL;'
                      'Trusted_Connection=yes;')

<h1>Postgres</h1>

In [1822]:
from psycopg2 import connect, sql, DatabaseError
import psycopg2.extras as extras

In [1823]:
from psycopg2.extensions import register_adapter, AsIs
register_adapter(np.int32, AsIs)
register_adapter(np.int64, AsIs)

In [1824]:
# Connect
try:
    conn = connect (
        dbname = "documentskm",
        user = "postgres",
        host = "localhost",
        password = "password"
    )
    cursor = conn.cursor()
except Exception as err:
    cursor = None
    print("Psycopg2 error:", err)
    
# Check if the connection was valid
if cursor != None:
    print("Connection successful")

Connection successful


In [1825]:
def execute_values(conn, df, table):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

<h1>Id sequence will be out of sync</h1>

select setval('mark_linked_docs_id_seq', (select max(id) from mark_linked_docs));

select setval('general_data_points_id_seq', (select max(id) from general_data_points));

<h2>If you need to drop all tables</h2>

drop schema public cascade;
create schema public;

<h1>Должности</h1>

In [1826]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_должностей', db_conn)

In [1827]:
df.at[13, "наим_должн"] = "инж. 3 к"

In [1828]:
df = df.rename(columns={"должность": "id",
                        "наим_должн": "name"
                       })

In [1829]:
cursor.execute(open("sql/25.sql", "r").read())
conn.commit()

In [1830]:
execute_values(conn, df, "positions")

execute_values() done


<h1>Отделы</h1>

In [1831]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_отделов', db_conn)

In [1832]:
df = df.drop(["начальник", "шифр_отд"], axis=1)

In [1833]:
df["наименование"] = df["наименование"].str.strip()

In [1834]:
df = df[df["код_отд"] != 40]
df = df[df["код_отд"] != 37]
df = df[df["код_отд"] != 36]
df = df[df["код_отд"] != 35]
df = df[df["код_отд"] != 34]
df = df[df["код_отд"] != 33]
df = df[df["код_отд"] != 32]
df = df[df["код_отд"] != 31]
df = df[df["код_отд"] != 18]
df = df[df["код_отд"] != 27]

In [1835]:
df.at[0, "наименование"] = "СО-2б"

In [1836]:
df.at[1, "наименование"] = "Машиностроительный"

In [1837]:
df = df[df["код_отд"] != 29]

In [1838]:
df.at[30, "наименование"] = "СО-2а"

In [1839]:
df = df.rename(columns={"код_отд": "id",
                        "наименование": "name"
                       })

In [1840]:
cursor.execute(open("sql/30.sql", "r").read())
conn.commit()

In [1841]:
execute_values(conn, df, "departments")

execute_values() done


<h1>Исполнители</h1>

In [1842]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_исполн', db_conn)

In [1843]:
df = df.dropna()

In [1844]:
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(40, 1)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(37, 1)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(36, 1)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(35, 26)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(34, 1)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(33, 26)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(32, 1)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(31, 26)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(18, 9)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(27, 26)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(29, 3)

In [1845]:
df = df[df["код_отд"] != 0]

In [1846]:
df["должность"] = df["должность"].str.strip()

In [1847]:
df.loc[df["должность"] == "ген.дир", 'тип_должн'] = 1
df.loc[df["должность"] == "гл.инж", 'тип_должн'] = 2
df.loc[df["должность"] == "зам.гл.инж", 'тип_должн'] = 3
df.loc[df["должность"] == "гл.стр", 'тип_должн'] = 4
df.loc[df["должность"] == "гл.арх", 'тип_должн'] = 5
df.loc[df["должность"] == "ГИП", 'тип_должн'] = 6
df.loc[df["должность"] == "нач.отд", 'тип_должн'] = 7
df.loc[df["должность"] == "зам.нач.отд", 'тип_должн'] = 8
df.loc[df["должность"] == "гл.спец.", 'тип_должн'] = 9
df.loc[df["должность"] == "Гл. спец.", 'тип_должн'] = 9
df.loc[df["должность"] == "зав.гр", 'тип_должн'] = 10
df.loc[df["должность"] == "Зав.гр", 'тип_должн'] = 10
df.loc[df["должность"] == "зав. Гр", 'тип_должн'] = 10
df.loc[df["должность"] == "вед.инж", 'тип_должн'] = 11
df.loc[df["должность"] == "вед.инж.", 'тип_должн'] = 11
df.loc[df["должность"] == "инж. 1 к", 'тип_должн'] = 12
df.loc[df["должность"] == "Инж. 1к", 'тип_должн'] = 12
df.loc[df["должность"] == "инженер1к", 'тип_должн'] = 12
df.loc[df["должность"] == "инж. 2 к", 'тип_должн'] = 13
df.loc[df["должность"] == "инж. 3 к", 'тип_должн'] = 14
df.loc[df["должность"] == "Инж. 3 к", 'тип_должн'] = 14
df.loc[df["должность"] == "инж3кат", 'тип_должн'] = 14
df.loc[df["должность"] == "инженер", 'тип_должн'] = 15
df.loc[df["должность"] == "инж", 'тип_должн'] = 15
df.loc[df["должность"] == "техник", 'тип_должн'] = 16
df.loc[df["должность"] == "чертежн", 'тип_должн'] = 17

In [1848]:
df["фио"] = df["фио"].str.strip()

In [1849]:
df = df.drop_duplicates(subset=["тип_должн", "фио", "код_отд"])

In [1850]:
df = df[df["тип_должн"] != 0]

In [1851]:
df = df.drop(["шифр", "должность"], axis=1)

In [1852]:
df = df.rename(columns={"код": "id",
                        "тип_должн": "position_id",
                        "фио": "name",
                        "код_отд": "department_id"
                       })

In [1853]:
cursor.execute(open("sql/26.sql", "r").read())
conn.commit()

In [1854]:
execute_values(conn, df, "employees")

execute_values() done


In [1855]:
df["id"].to_csv("employee_ids.csv")

<h1>Пользователи</h1>

In [1856]:
#df = pd.DataFrame({'id': [1, 2], 'login': ["1", "2"], 'password': [bcrypt.hashpw("1", bcrypt.gensalt()), bcrypt.hashpw("2", bcrypt.gensalt())], 'employee_id': [11, 27]})

In [1857]:
cursor.execute(open("sql/xx_users.sql", "r").read())
conn.commit()

In [1858]:
#execute_values(conn, df, "users")

<h1>Условия эксплуатации</h1>

In [1859]:
df1 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["неагрессивная", "слабоагрессивная", "среднеагрессивная", "сильноагрессивная"]})

In [1860]:
df2 = pd.DataFrame({'id': [1, 2, 3], 'name': ["внутри помещения", "снаружи помещения", "в жидкостях"]})

In [1861]:
df3 = pd.DataFrame({'id': [1, 2, 3, 4, 5], 'name': ["нет газов (вода)", "газы группы А (кислоты)", "газы группы B, C и D (щелочи)", "бензин", "масло"]})

In [1862]:
df4 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["сталь без покрытия", "оцинкованная сталь", "сталь горячего цинкования", "сталь с газотермическим напылением цинка"]})

In [1863]:
df5 = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 'name': ["ПФ", "АУ", "ЭЭ", "МА", "МБ", "НЦ", "ФФ", "ПА", "ПЭ", "ПБ", "ПВ", "СВ", "ПУ", "ЭП", "ПС", "КО"]})

In [1864]:
df6 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["отсутствуют", "сдвигоустойчивые соединения", "фланцевые соединения", "сдвигоустойчивые и фланцевые соединения"]})

In [1865]:
cursor.execute(open("sql/y_1.sql", "r").read())
cursor.execute(open("sql/y_2.sql", "r").read())
cursor.execute(open("sql/y_3.sql", "r").read())
cursor.execute(open("sql/y_4.sql", "r").read())
cursor.execute(open("sql/y_5.sql", "r").read())
cursor.execute(open("sql/y_6.sql", "r").read())
conn.commit()

In [1866]:
execute_values(conn, df1, "env_aggressiveness")
execute_values(conn, df2, "operating_areas")
execute_values(conn, df3, "gas_groups")
execute_values(conn, df4, "construction_materials")
execute_values(conn, df5, "paintwork_types")
execute_values(conn, df6, "high_tensile_bolts_types")

execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done


<h1>Контроль сварки</h1>

In [1867]:
df = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["контроль не требуется", "контроль опрыскиванием керосином", "контроль ультразвуком", "контроль гамма-излучением"]})

In [1868]:
cursor.execute(open("sql/yy_1.sql", "r").read())
conn.commit()

In [1869]:
execute_values(conn, df, "welding_control")

execute_values() done


<h1>Типы документов</h1>

In [1870]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5], 'code': ["л.", "СМ", "ВМП", "СМС", "РР"], 'name': ["Листы основного комплекта", "Спецификация металлопроката", "Ведомость металлоконструкций по видам профилей", "Сводная спецификация металлопроката", "Расчет металлоконструкций"]})

In [1871]:
cursor.execute(open("sql/x.sql", "r").read())
conn.commit()

In [1872]:
execute_values(conn, df, "doc_types")

execute_values() done


<h1>Степени очистки антикоррозионной защиты</h1>

In [1873]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Степени_оч_аз', db_conn)

In [1874]:
df["Наим_оч"] = df["Наим_оч"].str.strip()

In [1875]:
df = df.rename(columns={"Степень_оч": "id",
                        "Наим_оч": "name"
                       })

In [1876]:
cursor.execute(open("sql/44.sql", "r").read())
conn.commit()

In [1877]:
execute_values(conn, df, "corr_prot_cleaning_degrees")

execute_values() done


<h1>Варианты антикоррозионной защиты </h1>

In [1878]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Варианты_аз', db_conn)

In [1879]:
df = df.replace("", np.nan)

In [1880]:
df = df.dropna()

In [1881]:
df = df.drop_duplicates()

In [1882]:
df = df.drop(["Статус_аз"], axis=1)

In [1883]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1884]:
df = df.drop_duplicates(subset=["Зона_экс", "Группа_газов", "Агресс_ср", "Матер_к", "Тип_лп", "Группа_лп", "Стойкость_лп"])

In [1885]:
df = df.rename(columns={"Зона_экс": "operating_zone",
                        "Группа_газов": "gas_group",
                        "Агресс_ср": "env_aggressiveness",
                        "Матер_к": "material",
                        "Тип_лп": "paintwork_type",
                        "Группа_лп": "paintwork_group",
                        "Стойкость_лп": "paintwork_durability",
                        "Кол_слоев_лп": "paintwork_num_of_layers",
                        "Толщ_лпгр": "paintwork_primer_thickness",
                        "Кол_слоев_гр": "primer_num_of_layers",
                        "Степень_оч": "cleaning_degree_id"
                       })

In [1886]:
cursor.execute(open("sql/1.sql", "r").read())
conn.commit()

In [1887]:
execute_values(conn, df, "corr_prot_variants")

execute_values() done


<h1>Виды конструкций</h1>

In [1888]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_видов_к', db_conn)

In [1889]:
df["Имя_к"] = df["Имя_к"].str.strip()

In [1890]:
df = df[df["Имя_к"].isna() == False]

In [1891]:
df = df.rename(columns={"Шифр_к": "id",
                        "Имя_к": "name"
                       })

In [1892]:
df.loc[df["id"] == 2, "name"] = "Двухветвевые колонны одноэтажных производственных зданий и крановых эстакад"
df.loc[df["id"] == 9, "name"] = "Подкраново-подстропильные фермы"
df.loc[df["id"] == 34, "name"] = "Переплеты, витражи из профильных труб с деталями крепления"

In [1893]:
cursor.execute(open("sql/22.sql", "r").read())
conn.commit()

In [1894]:
execute_values(conn, df, "construction_types")

execute_values() done


<h1>Подвиды конструкций</h1>

In [1895]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_видов_пк', db_conn)

In [1896]:
df["Имя_пк"] = df["Имя_пк"].str.strip()
df["Расценка"] = df["Расценка"].str.strip()

In [1897]:
df = df.drop_duplicates()

In [1898]:
df = df.drop(["Шифр_пк"], axis=1)

In [1899]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1900]:
df = df.rename(columns={"Шифр_к": "type_id",
                        "Имя_пк": "name",
                        "Расценка": "valuation"
                       })

In [1901]:
df.loc[df["id"] == 11, "name"] = "высотой свыше 150 м, поставляемые отдельными элементами"
df.loc[df["id"] == 14, "name"] = "коробчатого сечения пролетом 12 м и более"
df.loc[df["id"] == 33, "name"] = "с шагом стропильных ферм 6 м"
df.loc[df["id"] == 34, "name"] = "с шагом стропильных ферм 12 м"
df.loc[df["id"] == 49, "name"] = "составного сечения из горячекатаных профилей, соединенных планками"
df.loc[df["id"] == 51, "name"] = "ветви из двутавров с парал. гранями полок, решетка из г/кат. уголков и швеллеров"
df.loc[df["id"] == 52, "name"] = "ветви из двутавров (ГОСТ 8239-72) или швеллеров и решетка из г/кат. уголков"
df.loc[df["id"] == 69, "name"] = "с поясами из тавров, стенкой из листовой стали, пролетом 6 м, массой 1 м,т:"
df.loc[df["id"] == 70, "name"] = "с поясами из тавров, стенкой из листовой стали, пролетом 12 м и более,т:"
df.loc[df["id"] == 71, "name"] = "из листовой стали пролетом 6 м, массой 1 м,т:"
df.loc[df["id"] == 72, "name"] = "из листовой стали пролетом 12 м, массой 1 м,т:"
df.loc[df["id"] == 83, "name"] = "пролетом 12 м, решетчатые из г/кат. и гнутых профилей, масса 1 м до 0.4 т"
df.loc[df["id"] == 84, "name"] = "пролетом 12 м, решетчатые из г/кат. и гнутых профилей, масса 1 м свыше 0.4 т"
df.loc[df["id"] == 85, "name"] = "пролетом 12 м, решетчатые сост. сечения из листовой стали, масса 1 м до 0.4 т"
df.loc[df["id"] == 86, "name"] = "пролетом 12 м, решетчатые сост. сечения из листовой стали, масса 1 м свыше 0.4 т"
df.loc[df["id"] == 87, "name"] = "составного сечения из листовой стали, масса 1 м,т:"
df.loc[df["id"] == 88, "name"] = "составного сечения из двух и более уголков или швеллеров, масса 1 м,т:"
df.loc[df["id"] == 89, "name"] = "из двутавров с параллельными гранями полок, масса 1 м,т:"
df.loc[df["id"] == 90, "name"] = "составного сечения из листовой стали с верхним опиранием настила, масса марки,т:"
df.loc[df["id"] == 91, "name"] = "составного сечения при пониженном опирании настила, масса марки,т:"
df.loc[df["id"] == 92, "name"] = "из двутавров с парал. гранями полок с верхним опиранием настила, масса до 1 т"
df.loc[df["id"] == 93, "name"] = "из двутавров с парал. гранями полок с верхним опиранием настила, масса свыше 1 т"
df.loc[df["id"] == 94, "name"] = "из двутавров с парал. гранями полок с пониж. опиранием настила, масса до 1 т"
df.loc[df["id"] == 95, "name"] = "из двутавров с парал. гранями полок с пониж. опиранием настила, масса свыше 1 т"
df.loc[df["id"] == 113, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м до 0.5 т"
df.loc[df["id"] == 114, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м до 1 т"
df.loc[df["id"] == 115, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м свыше 1 т"

In [1902]:
cursor.execute(open("sql/24.sql", "r").read())
conn.commit()

In [1903]:
execute_values(conn, df, "construction_subtypes")

execute_values() done


<h1>Типовые наименования листов</h1>

In [1904]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_листов_н', db_conn)

In [1905]:
df["ш_листа"] = df["ш_листа"].str.strip()

In [1906]:
df = df.rename(columns={"ш_номер": "id",
                        "ш_листа": "name"
                       })

In [1907]:
cursor.execute(open("sql/27.sql", "r").read())
conn.commit()

In [1908]:
execute_values(conn, df, "sheet_names")

execute_values() done


<h1>Виды ссылочных документов</h1>

In [1909]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_ссыл_в', db_conn)

In [1910]:
df["Наим_вида"] = df["Наим_вида"].str.strip()
df["Наим_вида"] = df["Наим_вида"].str.capitalize()

In [1911]:
df = df.drop_duplicates(subset=["Наим_вида"])

In [1912]:
df = df.rename(columns={"Вид_сд": "id",
                        "Наим_вида": "name"
                       })

In [1913]:
cursor.execute(open("sql/40.sql", "r").read())
conn.commit()

In [1914]:
execute_values(conn, df, "linked_doc_types")

execute_values() done


<h1>Ссылочные документы</h1>

In [1915]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_ссыл', db_conn)

In [1916]:
df = df.replace("", np.nan)

In [1917]:
df = df.dropna()

In [1918]:
df = df.drop_duplicates(subset=["Шифр_сд"])

In [1919]:
df = df.reset_index(drop=True)

In [1920]:
df["Шифр_сд"] = df["Шифр_сд"].str.strip()
df["Обозн_сд"] = df["Обозн_сд"].str.strip()
df["Наим_сд"] = df["Наим_сд"].str.strip()

In [1921]:
for i in range(0, len(df)):
    arr = df["Наим_сд"][i].split()
    new_str = ' '.join(arr)
    df.at[i, "Наим_сд"] = new_str

In [1922]:
df["Наим_сд"] = df["Наим_сд"].str.replace("возду ха", "воздуха")
df["Наим_сд"] = df["Наим_сд"].str.replace("Материриалы", "Материалы")
df["Наим_сд"] = df["Наим_сд"].str.replace("рекоментации", "рекомендации")
df["Наим_сд"] = df["Наим_сд"].str.replace("рас пашные", "распашные")
df["Наим_сд"] = df["Наим_сд"].str.replace("металличе ческих", "металлических")
df["Наим_сд"] = df["Наим_сд"].str.replace("металличе ских", "металлических")
df["Наим_сд"] = df["Наим_сд"].str.replace("электри ческие", "электрические")
df["Наим_сд"] = df["Наим_сд"].str.replace("кра ны", "краны")
df["Наим_сд"] = df["Наим_сд"].str.replace("примене нием", "применением")
df["Наим_сд"] = df["Наим_сд"].str.replace("обслуживани сосудов", "обслуживания сосудов")
df["Наим_сд"] = df["Наим_сд"].str.replace("пе реплетами", "переплетами")
df["Наим_сд"] = df["Наим_сд"].str.replace("за полнением", "заполнением")
df["Наим_сд"] = df["Наим_сд"].str.replace("линй", "линий")
df["Наим_сд"] = df["Наим_сд"].str.replace("мосто вые", "мостовые")
df["Наим_сд"] = df["Наим_сд"].str.replace("покрыти ем", "покрытием")
df["Наим_сд"] = df["Наим_сд"].str.replace("со кращенному", "сокращенному")
df["Наим_сд"] = df["Наим_сд"].str.replace("покрытияпролетами", "покрытия пролетами")
df["Наим_сд"] = df["Наим_сд"].str.replace("широкополочныых", "широкополочных")
df["Наим_сд"] = df["Наим_сд"].str.replace("тав ров", "тавров")
df["Наим_сд"] = df["Наим_сд"].str.replace("зда ний", "зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("ремонтно -монтажных", "ремонтно-монтажных")
df["Наим_сд"] = df["Наим_сд"].str.replace("пок рытия", "покрытия")
df["Наим_сд"] = df["Наим_сд"].str.replace("профилирован ным", "профилированным")
df["Наим_сд"] = df["Наим_сд"].str.replace("стеклои минераловатных", "стекло и минераловатных")
df["Наим_сд"] = df["Наим_сд"].str.replace("окон,дверей", "окон, дверей")
df["Наим_сд"] = df["Наим_сд"].str.replace("пром.зданий", "пром. зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("строитель ные", "строительные")
df["Наим_сд"] = df["Наим_сд"].str.replace("сталь ные", "стальные")
df["Наим_сд"] = df["Наим_сд"].str.replace("наруж ного", "наружного")
df["Наим_сд"] = df["Наим_сд"].str.replace("ветров.район.", "ветров. район.")
df["Наим_сд"] = df["Наим_сд"].str.replace("выше и ниже -40 град.с.  ше и ниже -40 град.с.", "выше -40 град.с. и ниже -40 град.с.")
df["Наим_сд"] = df["Наим_сд"].str.replace("алю миниевых", "алюминиевых")
df["Наим_сд"] = df["Наим_сд"].str.replace("зданий,промышленных", "зданий, промышленных")
df["Наим_сд"] = df["Наим_сд"].str.replace("предприятий.шарнирные", "предприятий. Шарнирные")
df["Наим_сд"] = df["Наим_сд"].str.replace("связей.Чертежи", "связей. Чертежи")
df["Наим_сд"] = df["Наим_сд"].str.replace("сталь ными", "стальными")
df["Наим_сд"] = df["Наим_сд"].str.replace("унифици рованных", "унифицированных")
df["Наим_сд"] = df["Наим_сд"].str.replace("уз лы", "узлы")
df["Наим_сд"] = df["Наим_сд"].str.replace("железобе тонных", "железобетонных")
df["Наим_сд"] = df["Наим_сд"].str.replace("метал лопроката", "металлопроката")
df["Наим_сд"] = df["Наим_сд"].str.replace("выпу ску", "выпуску")
df["Наим_сд"] = df["Наим_сд"].str.replace("откры вающиеся", "открывающиеся")
df["Наим_сд"] = df["Наим_сд"].str.replace("светоаэра ционных", "светоаэрационных")
df["Наим_сд"] = df["Наим_сд"].str.replace("проектирования.Рабочие", "проектирования. Рабочие")
df["Наим_сд"] = df["Наим_сд"].str.replace("черте жи", "чертежи")
df["Наим_сд"] = df["Наим_сд"].str.replace("опорн.кранами", "опорн. кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("опорынми кранами", "опорными кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("несущ.конструкций", "несущ. конструкций")
df["Наим_сд"] = df["Наим_сд"].str.replace("подв.кранами", "подв. кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("кровлей,оборудованных", "кровлей, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("одноэт.произв", "одноэт. произв")
df["Наим_сд"] = df["Наим_сд"].str.replace("произв.зданий", "произв. зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("кровлей,оборудованных", "кровлей, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("специальн.назначения", "специальн. назначения")
df["Наим_сд"] = df["Наим_сд"].str.replace("зданий,оборудованных", "зданий, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("две рей", "дверей")

In [1923]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1924]:
df = df.rename(columns={"Шифр_сд": "code",
                        "Вид_сд": "type_id",
                        "Обозн_сд": "designation",
                        "Наим_сд": "name"
                       })

In [1925]:
df = df.drop_duplicates(subset=["name"])

In [1926]:
df.loc[df["name"].str.endswith('.') == False, "name"] = df[df["name"].str.endswith('.') == False]["name"] + '.'

In [1927]:
cursor.execute(open("sql/39.sql", "r").read())
conn.commit()

In [1928]:
execute_values(conn, df, "linked_docs")

execute_values() done


<h1>Категории производства по пожарной и взрывопожарной опасности</h1>

In [1929]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Категории_пр', db_conn)

In [1930]:
df = df.replace("", np.nan)

In [1931]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1932]:
df = df.rename(columns={"Категория": "category",
                        "Название": "name",
                        "Текст_од": "description"
                       })

In [1933]:
df = df.where(pd.notnull(df), None)

In [1934]:
cursor.execute(open("sql/5.sql", "r").read())
conn.commit()

In [1935]:
execute_values(conn, df, "fire_hazard_categories")

execute_values() done


<h1>Проекты</h1>

In [1936]:
df = pd.read_sql_query('SELECT * FROM Pro_t_2.dbo.Марки', db_conn)
df.shape

(4112, 33)

In [1937]:
df = df.dropna(subset=["код_марки", "обозн", "комплекс", "объект"])

In [1938]:
e_ids = pd.read_csv("employee_ids.csv")
e_ids = e_ids.drop(["Unnamed: 0"], axis=1)
e_ids = e_ids.values.flatten().tolist()
df.loc[df["гл_спец"].isin(e_ids) == False, "гл_спец"] = None
df.loc[df["рук_гр"].isin(e_ids) == False, "рук_гр"] = None
df.loc[df["н_контр"].isin(e_ids) == False, "н_контр"] = None

In [1939]:
df = df[df["обозн"].str.contains("демонтаж") == False]
df = df[df["обозн"].str.contains("Мновый") == False]
df["обозн"] = df["обозн"].str.replace(",", ".")

In [1940]:
df2 = pd.DataFrame(df["обозн"].str.split('-КМ', 1).tolist(),
                                 columns = ['first','code'])

In [1941]:
df = df.reset_index()
df2 = df2.join(df)

In [1942]:
df2 = df2.dropna(subset=["code"])

In [1943]:
df2["code"] = "КМ" + df2["code"]

In [1944]:
df3 = pd.DataFrame(df2["first"].str.split('.', 1).tolist(),
                                 columns = ['baseSeries','middle'])

In [1945]:
df2 = df2.reset_index()
df3 = df3.reset_index()
df4 = df2.join(df3[["baseSeries", "middle"]])

In [1946]:
df4 = df4.drop(["first", "index"], axis=1)
df4 = df4.rename(columns={"код_марки": "id"})

In [1947]:
df5 = pd.DataFrame(df4["middle"].fillna("").str.split('.', 1).tolist(),
                                 columns = ['node','subnode'])

In [1948]:
df6 = pd.DataFrame(df5["subnode"].fillna("").str.split('-', 1).tolist(),
                                 columns = ['subnode_part','node_part'])

In [1949]:
df5 = df5.reset_index()
df6 = df6.reset_index()
df7 = df5.join(df6[["subnode_part", "node_part"]])

In [1950]:
df7["node_part"] = "-" + df7["node_part"]
df7["node2"] = df7["node"] + df7["node_part"].fillna("")

In [1951]:
df4 = df4.reset_index(drop=True)
df7 = df7.reset_index(drop=True)
df8 = df4.join(df7[["node2", "subnode_part"]])

In [1952]:
df8 = df8.rename(columns={"node2": "nodeCode", "subnode_part": "subnodeCode"})
df8 = df8.drop(["middle"], axis=1)

In [1953]:
df8[df8["baseSeries"] == "М32833"]

,level_0,code,id,обозн,комплекс,объект,код_отд,подп1,подп2,гл_спец,...,10ХСНД,ТекстЗдСм,ДопОбъемы,ВесПоВМП,ВыпЗдСм,ДопКод,ОЗ_марки,baseSeries,nodeCode,subnodeCode
376,379,КМ1,4610,М32833.07-КМ1,"ОАО ""ММК"". Доменный цех. Печь №1. Техперевоору...",Шихтоподача. Демонтаж.,1.0,258.0,89.0,139.0,...,False,None,None,0.000000,0.0,None,0,М32833,07,
402,405,КМ5,4721,М32833.07-КМ5,"ПАО ""ММК"". Доменный цех. Печь №1. Техперевоору...",Шихтоподача. Реконструкция приемных бункеров,1.0,258.0,89.0,139.0,...,False,Разработать сметную документацию к чертежам М3...,None,550.030029,0.0,None,0,М32833,07,
409,412,КМ3,4646,М32833.01-КМ3,"ОАО ""ММК"". Доменный цех. Печь №1. Техперевоору...",Собственно печь. Площадка для выпуска верхнего...,1.0,258.0,89.0,139.0,...,False,Разработать сметную документацию к чертежам М3...,"1, К=1,35 \r\n2. Демонтаж конструкций в лом",21.520000,0.0,None,0,М32833,01,
412,415,КМ2,4649,М32833.01-КМ2,"ОАО ""ММК"". Доменный цех. Печь №1. Техперевоору...",Собственно печь. Временная опорная система печ...,1.0,258.0,89.0,139.0,...,False,Разработать сметную документацию к чертежам М3...,1. После монтажа кожуха - данные конструкции д...,14.690000,0.0,None,0,М32833,01,
413,416,КМ4,4651,М32833.01-КМ4,"ОАО ""ММК"". Доменный цех. Печь №1. Техперевоору...",Собственно печь. Настил и ограждение колошнико...,1.0,258.0,89.0,139.0,...,False,Разработать сметную документацию к чертежам М3...,"К=1,35",33.000000,0.0,None,0,М32833,01,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,774,КМ65,5057,М32833.04-КМ65,"ПАО ""ММК"". Доменный цех. Печь № 1. Техперевоор...","Литейные дворы. Площадка на\r\nотм. +9,800",1.0,258.0,536.0,139.0,...,False,Разработать сметную документацию к чертежам М3...,1. Коэффициент к нормам затрат труда на услови...,1.754000,0.0,None,0,М32833,04,
773,778,КМ11 изм.1,5061,М32833.03-КМ11 изм.1,"ПАО ""ММК"". Доменный цех. Печь №1. Техперевоору...",Воздухонагреватели. Реконструкция площадок на ...,1.0,258.0,89.0,139.0,...,False,Разработать сметную документацию к чертежам М3...,None,10.091000,0.0,None,0,М32833,03,
778,783,КМ2.2,5067,М32833.05-КМ2.2,"ПАО ""ММК"". Доменный цех. Печь №1. \r\nТехперев...",Машинный зал. Фахверк и площадки. Оси 2н-1 и 4...,1.0,258.0,536.0,139.0,...,False,Разработать сметную документацию к чертежам М3...,1. В данной рабочей документации предусмотрен ...,2.353000,0.0,None,0,М32833,05,
818,823,КМ3.1 доп смета,5110,М32833.11-КМ3.1 доп смета,"ПАО ""ММК"". Доменный цех. Печь №1. Техперевоору...",Пост управления печью. Лестница на пост управл...,1.0,258.0,89.0,139.0,...,False,"Заменить расценку согласно письма УКСа ПАО ""М...",None,0.000000,0.0,None,0,М32833,11,


In [1954]:
enc = OrdinalEncoder()
res = enc.fit_transform(np.array(df8["baseSeries"]).reshape(-1, 1))

In [1955]:
s = pd.DataFrame(pd.Series(res.flatten()) + 1)
s = s.rename(columns={0: "project_id"})

In [1956]:
df9 = df8.join(s)

In [1957]:
projects_df = df9[["baseSeries", "комплекс", "project_id"]]

In [1958]:
projects_df = projects_df.drop_duplicates(subset=["project_id"])

In [1959]:
projects_df = projects_df.rename(columns={"baseSeries": "base_series", "комплекс": "name", "project_id": "id"})

In [1960]:
cursor.execute(open("sql/xx_projects.sql", "r").read())
conn.commit()

In [1961]:
execute_values(conn, projects_df, "projects")

execute_values() done


<h1>Узлы</h1>

In [1962]:
df10 = pd.DataFrame(df9["объект"].str.split('.', 1).tolist(),
                                 columns = ['f_p','o_p'])

In [1963]:
df10 = df10.fillna("")

In [1964]:
df11 = df9.join(df10)

In [1965]:
df11["node_feature"] = df11["baseSeries"] + df11["nodeCode"]

In [1966]:
enc = OrdinalEncoder()
res = enc.fit_transform(np.array(df11["node_feature"]).reshape(-1, 1))

In [1967]:
s = pd.DataFrame(pd.Series(res.flatten()) + 1)
s = s.rename(columns={0: "node_id"})

In [1968]:
df12 = df11.join(s)

In [1969]:
nodes_df = df12[["node_id", "f_p", "project_id", "nodeCode"]]

In [1970]:
chiefEngineerIds = [92, 123, 144, 169, 201, 270, 305, 311, 368, 370, 396, 400, 443, 461, 462, 604, 608]

In [1971]:
nodes_df["chief_engineer"] = random.choices(chiefEngineerIds, k=len(nodes_df))

C:\Users\sigla\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [1972]:
nodes_df = nodes_df.rename(columns={"node_id": "id", "f_p": "name", "nodeCode": "code", "chief_engineer": "chief_engineer_id"})

In [1973]:
nodes_df = nodes_df.drop_duplicates(subset=["id"])

In [1974]:
cursor.execute(open("sql/xx_nodes.sql", "r").read())
conn.commit()

In [1975]:
execute_values(conn, nodes_df, "nodes")

execute_values() done


<h1>Подузлы</h1>

In [1976]:
df13 = pd.DataFrame(df12["o_p"].str.split('.', 1).tolist(),
                                 columns = ['s_p','m_p'])

In [1977]:
df13 = df13.fillna("")

In [1978]:
df13["s_p"] = df13["s_p"].str.strip()
df13["m_p"] = df13["m_p"].str.strip()

In [1979]:
df14 = df12.join(df13)

In [1980]:
df14["subnode_feature"] = df11["node_feature"] + df11["subnodeCode"]

In [1981]:
enc = OrdinalEncoder()
res = enc.fit_transform(np.array(df14["subnode_feature"]).reshape(-1, 1))

In [1982]:
s = pd.DataFrame(pd.Series(res.flatten()) + 1)
s = s.rename(columns={0: "subnode_id"})

In [1983]:
df15 = df14.join(s)

In [1984]:
subnodes_df = df15[["subnode_id", "node_id", "subnodeCode", "s_p"]]

In [1985]:
subnodes_df = subnodes_df.rename(columns={"subnode_id": "id", "s_p": "name", "subnodeCode": "code"})

In [1986]:
subnodes_df = subnodes_df.drop_duplicates(subset=["id"])

In [1987]:
cursor.execute(open("sql/xx_subnodes.sql", "r").read())
conn.commit()

In [1988]:
execute_values(conn, subnodes_df, "subnodes")

execute_values() done


<h1>Марки</h1>

In [1989]:
df15 = df15.drop(["обозн", "комплекс", "объект", "level_0", "ОЗ_марки"], axis=1)

In [1990]:
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("минус ", "-")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("плюс ", "")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("+", "")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("- 39", "-39")
df15 = df15[df15["t_эксплуат"] != "-"]
df15 = df15[df15["t_эксплуат"] != "минус"]
df15["t_эксплуат"] = df15["t_эксплуат"].astype("float64")

In [1991]:
df15["об_марки"] = df15["об_марки"].str.strip()
df15["об_марки"] = df15["об_марки"].str.replace("KM", "КМ")
df15 = df15[df15["об_марки"] != "по"]

In [1992]:
df15 = df15.drop(["об_марки"], axis=1)

In [1993]:
df15 = df15.drop(["project_id", "f_p", "o_p", "s_p", "subnode_feature", "node_feature", "subnodeCode"], axis=1)

In [1994]:
df15 = df15.drop(["baseSeries", "nodeCode", "node_id"], axis=1)

In [1995]:
df15["прим"] = df15["прим"].str.strip()
df15["прим"] = df15["прим"].str.replace("\r\n       ", " ")

In [1996]:
df15 = df15.drop(["ДопКод"], axis=1)

In [1997]:
df15["категория_пр"] = 3

In [1998]:
df15[["зона_эксплуат", "группа_газов", "агрессивность", "материал", "тип_ЛКМ"]].drop_duplicates()

,зона_эксплуат,группа_газов,агрессивность,материал,тип_ЛКМ
0,2,1.0,1,0,ПФ
19,2,1.0,2,0,ПФ
25,1,1.0,2,0,ПФ
27,1,1.0,2,0,АУ
35,2,1.0,0,0,ПФ
70,1,1.0,0,0,ПФ
168,1,1.0,1,0,ПФ
649,2,0.0,1,0,ПФ
684,1,0.0,0,0,ПФ
814,1,4.0,1,0,ПФ


In [1999]:
df15["агрессивность"] += 1
df15["группа_газов"] += 1
df15["материал"] += 1
df15["вп_болты"] += 1

# Part
df15.loc[df15["тип_ЛКМ"] == "ПФ", "тип_ЛКМ"] = 1
df15.loc[df15["тип_ЛКМ"] == "АУ", "тип_ЛКМ"] = 2
df15.loc[df15["тип_ЛКМ"] == "ЭЭ", "тип_ЛКМ"] = 3
df15.loc[df15["тип_ЛКМ"] == "МА", "тип_ЛКМ"] = 4
df15.loc[df15["тип_ЛКМ"] == "ПВ", "тип_ЛКМ"] = 11

In [2000]:
df15 = df15.drop_duplicates(subset=["code", "subnode_id"])

In [2001]:
df15_operating_cond = df15.copy()

In [2002]:
df15 = df15.rename(columns={"код_отд": "department_id",
                            "подп1": "signed1_id",
                            "подп2": "signed2_id",
                            "гл_спец": "chief_specialist_id",
                            "рук_гр": "group_leader_id",
                            "н_контр": "main_builder_id",
                            "дата_выд": "issued_date",
                            "кол_томов": "num_of_volumes",
                            "дата_ред": "edited_date",
                            "коэф_надежн": "safety_coeff",
                            "t_эксплуат": "operating_temp",
                            "зона_эксплуат": "operating_zone",
                            "группа_газов": "gas_group",
                            "агрессивность": "aggressiveness",
                            "материал": "material",
                            "тип_ЛКМ": "paintwork_type",
                            "прим": "note",
                            "категория_пр": "fire_hazard_category_id",
                            "вп_болты": "high_tensile_bolts",
                            "п_транспорт": "p_transport",
                            "п_площадки": "p_site",
                            "10ХСНД": "xcnd",
                            "ТекстЗдСм": "text_3d_estimate",
                            "ДопОбъемы": "add_volumes",
                            "ВесПоВМП": "vmp_weight",
                            "ВыпЗдСм": "impl_3d_estimate",
                            "m_p": "name"
                       })

In [2003]:
df15["issued_date"] = df15["issued_date"].astype(object).where(df15["issued_date"].notnull(), None)
df15["edited_date"] = df15["edited_date"].astype(object).where(df15["edited_date"].notnull(), None)
df15 = df15.where(pd.notnull(df15), None)

In [2004]:
df15["chief_specialist_id"] = df15["chief_specialist_id"].replace(0, None)
df15["group_leader_id"] = df15["group_leader_id"].replace(0, None)
df15["main_builder_id"] = df15["main_builder_id"].replace(0, None)

In [2005]:
df16 = df15.reset_index(drop=True)
pd.DataFrame(df16.iloc[0, :].values.reshape(1, -1), columns=df16.columns.tolist())

,code,id,department_id,signed1_id,signed2_id,chief_specialist_id,group_leader_id,main_builder_id,issued_date,num_of_volumes,...,high_tensile_bolts,p_transport,p_site,xcnd,text_3d_estimate,add_volumes,vmp_weight,impl_3d_estimate,name,subnode_id
0,КМ1,3173,1,258,272,139,None,92,2013-02-21,0,...,1,False,False,False,Разработать сметную документацию к чертежам М3...,"Работы ведутся на высоте до 4,430 м. от уровня...",3.95,0,Дооборудование. Ворота,26


In [2006]:
cursor.execute(open("sql/7.sql", "r").read())
conn.commit()

In [2007]:
execute_values(conn, df16, "marks")

execute_values() done


In [2008]:
df16["id"].to_csv("mark_ids.csv")

<h1>Условия эксплуатации марки</h1>

In [2009]:
df_op_cond = df15_operating_cond.loc[:, ["id", "коэф_надежн", "агрессивность", "t_эксплуат", "зона_эксплуат", "группа_газов", "материал", "тип_ЛКМ", "вп_болты"]]

In [2010]:
df_op_cond["t_эксплуат"] = df_op_cond["t_эксплуат"].fillna(-34)

In [2011]:
df_op_cond = df_op_cond.rename(columns={"id": "mark_id",
                            "коэф_надежн": "safety_coeff",
                            "t_эксплуат": "temperature",
                            "зона_эксплуат": "operating_area_id",
                            "группа_газов": "gas_group_id",
                            "агрессивность": "env_aggressiveness_id",
                            "материал": "construction_material_id",
                            "тип_ЛКМ": "paintwork_type_id",
                            "вп_болты": "high_tensile_bolts_type_id"
                       })

In [2012]:
cursor.execute(open("sql/y_mark_operating_conditions.sql", "r").read())
conn.commit()

In [2013]:
execute_values(conn, df_op_cond, "mark_operating_conditions")

execute_values() done


<h1>Выпуски спецификаций</h1>

In [2014]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.ВыпСпец', db_conn)

In [2015]:
df = df.replace("", np.nan)

In [2016]:
df = df.drop_duplicates(subset=["код_марки", "выпуск"])

In [2017]:
df = df.drop(["вкл_вып"], axis=1)

In [2018]:
df["текущий"] = df["текущий"].map(lambda x: True if (x == 1) else False)

In [2019]:
df.insert(0, 'id', range(1, len(df) + 1))

In [2020]:
df = df.rename(columns={"код_марки": "mark_id",
                        "выпуск": "num",
                        "дата_созд": "created_date",
                        "текущий": "is_current",
                        "прим": "note"
                       })

In [2021]:
df["created_date"] = df["created_date"].astype(object).where(df["created_date"].notnull(), datetime(2000, 1, 1))
df = df.where(pd.notnull(df), None)

In [2022]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()

In [2023]:
df = df[df["mark_id"].isin(m_ids)]

In [2024]:
cursor.execute(open("sql/3.sql", "r").read())
conn.commit()

In [2025]:
execute_values(conn, df, "specifications")

execute_values() done


In [2026]:
df.loc[:, ["id", "mark_id", "num"]].to_csv("specs.csv")

<h1>Листы</h1>

In [2027]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Листы', db_conn)

In [2028]:
df = df.replace("", np.nan)

In [2029]:
df = df[df["номер"].isna() == False]

In [2030]:
df["тип_док"] = df["тип_док"] + 1

In [2031]:
df["вып"] = df["вып"].map(lambda x: np.nan if x == 0 else x)

In [2032]:
df["пров"] = df["пров"].map(lambda x: np.nan if x == 0 else x)

In [2033]:
df["норм"] = df["норм"].map(lambda x: np.nan if x == 0 else x)

In [2034]:
df = df.drop(["шифр_вып", "шифр_пров", "шифр_норм"], axis=1)

In [2035]:
df["формат"] = df["формат"].map(lambda x: 1.0 if np.isnan(x) else x)

In [2036]:
df = df.drop_duplicates(subset=["код_марки", "номер", "тип_док"])

In [2037]:
df["название"] = df["название"].str.strip()

In [2038]:
for i in range(0, len(df)):
    try:
        arr = df["название"][i].split()
        new_str = ' '.join(arr)
        df.at[i, "название"] = new_str
    except Exception:
        pass

In [2039]:
df.insert(0, 'id', range(1, len(df) + 1))

In [2040]:
df = df.rename(columns={"код_марки": "mark_id",
                        "номер": "num",
                        "название": "name",
                        "формат": "form",
                        "вып": "creator_id",
                        "пров": "inspector_id",
                        "норм": "norm_contr_id",
                        "тип_док": "type_id",
                        "выпуск": "release_num",
                        "листов": "num_of_pages",
                        "прим": "note"
                       })

In [2041]:
df = df.where(pd.notnull(df), None)

In [2042]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()

In [2043]:
e_ids = pd.read_csv("employee_ids.csv")
e_ids = e_ids.drop(["Unnamed: 0"], axis=1)
e_ids = e_ids.values.flatten().tolist()

In [2044]:
df = df[df["mark_id"].isin(m_ids)]
df.loc[df["creator_id"].isin(e_ids) == False, "creator_id"] = None
df.loc[df["inspector_id"].isin(e_ids) == False, "inspector_id"] = None
df.loc[df["norm_contr_id"].isin(e_ids) == False, "norm_contr_id"] = None

In [2045]:
cursor.execute(open("sql/6.sql", "r").read())
conn.commit()

In [2046]:
execute_values(conn, df, "docs")

execute_values() done


<h1>Обозначения марок</h1>

In [2047]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Об_марок', db_conn)

In [2048]:
df = df.iloc[1:, :]
df = df.iloc[:-1, :]
df.loc[len(df) + 1] = ["КМД", "Конструкции металлические деталировочные"]

In [2049]:
df.insert(0, 'id', range(1, len(df) + 1))

In [2050]:
df = df.rename(columns={"об_марки": "designation",
                        "наим_марки": "name"
                       })

In [2051]:
cursor.execute(open("sql/9.sql", "r").read())
conn.commit()

In [2052]:
execute_values(conn, df, "mark_designations")

execute_values() done


<h1>Прилагаемые документы марки</h1>

In [2053]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Од_прил', db_conn)

In [2054]:
df = df.replace("", np.nan)

In [2055]:
df = df.drop_duplicates(subset=["код_марки", "обозн_прил"])

In [2056]:
df = df.where(pd.notnull(df), None)

In [2057]:
df.insert(0, 'id', range(1, len(df) + 1))

In [2058]:
df = df.rename(columns={"код_марки": "mark_id",
                        "наим_прил": "name",
                        "прим": "note",
                        "обозн_прил": "designation"
                       })

In [2059]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["mark_id"].isin(m_ids)]

In [2060]:
cursor.execute(open("sql/12.sql", "r").read())
conn.commit()

In [2061]:
execute_values(conn, df, "attached_docs")

execute_values() done


<h1>Согласования</h1>

In [2062]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Од_согл', db_conn)

In [2063]:
df = df.replace("", np.nan)

In [2064]:
df = df.drop(["отдел1", "отдел2", "отдел3", "отдел4", "отдел5", "отдел6", "отдел7"], axis=1)

In [2065]:
p1 = df.iloc[:, [0,1]]
p1 = p1.rename(columns={"исп1": "исп"})

p2 = df.iloc[:, [0,2]]
p2 = p2.rename(columns={"исп2": "исп"})

p3 = df.iloc[:, [0,3]]
p3 = p3.rename(columns={"исп3": "исп"})

p4 = df.iloc[:, [0,4]]
p4 = p4.rename(columns={"исп4": "исп"})

p5 = df.iloc[:, [0,5]]
p5 = p5.rename(columns={"исп5": "исп"})

p6 = df.iloc[:, [0,6]]
p6 = p6.rename(columns={"исп6": "исп"})

p7 = df.iloc[:, [0,7]]
p7 = p7.rename(columns={"исп7": "исп"})

In [2066]:
df = pd.concat([p1, p2, p3, p4, p5, p6, p7], ignore_index=True)

In [2067]:
df = df.dropna()
df = df[df["исп"] != 0]

In [2068]:
df = df.drop_duplicates()

In [2069]:
df = df.rename(columns={"код_марки": "mark_id",
                        "исп": "employee_id"
                       })

In [2070]:
m_ids = pd.read_csv("mark_ids.csv")
e_ids = pd.read_csv("employee_ids.csv")

In [2071]:
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
e_ids = e_ids.drop(["Unnamed: 0"], axis=1)

In [2072]:
m_ids = m_ids.values.flatten().tolist()
e_ids = e_ids.values.flatten().tolist()

In [2073]:
df = df[df["mark_id"].isin(m_ids)]
df = df[df["employee_id"].isin(e_ids)]

In [2074]:
cursor.execute(open("sql/13.sql", "r").read())
conn.commit()

In [2075]:
execute_values(conn, df, "mark_approvals")

execute_values() done


<h1>Ссылочные документы марки (id sequence is out of sync)</h1>

In [2076]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Од_ссыл', db_conn)

In [2077]:
df = df.replace("", np.nan)

In [2078]:
df = df.drop(["прим"], axis=1)

In [2079]:
df= df[df["шифр_сд"] != "1"]
df["шифр_сд"] = df["шифр_сд"].str.upper()

In [2080]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [2081]:
for i in range(1, 301):
    df = df.append(pd.Series(), ignore_index=True)

C:\Users\sigla\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [2082]:
df.loc[:, "код_марки"] = random.choices(m_ids, k=300)
df.loc[:, "шифр_сд"] = random.choices(range(1, 300), k=300)

In [2083]:
df = df.drop_duplicates(subset=["код_марки", "шифр_сд"])

In [2084]:
df.insert(0, 'id', range(1, len(df) + 1))

In [2085]:
df = df.rename(columns={"код_марки": "mark_id",
                        "шифр_сд": "linked_doc_id",
                        "прим": "note"
                       })

In [2086]:
cursor.execute(open("sql/14.sql", "r").read())
conn.commit()

In [2087]:
execute_values(conn, df, "mark_linked_docs")

execute_values() done


<h1>Разделы общих указаний</h1>

In [2088]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
                   'name': ["Разработано на основании",
                            "Разработано в соответствии",
                            "Перечень технических регламентов",
                            "Высотная отметка",
                            "Климатические условия",
                            "Конструктивные решения",
                            "Характеристики конструкций",
                            "Материал конструкций",
                            "Указания по разработке КМД и ППР",
                            "Указания по изготовлению и монтажу",
                            "Крепление элементов",
                            "Соединение элементов",
                            "Антикоррозионная защита",
                            "Огнезащита конструкций",
                            "Перечень скрытых видов работ",
                            "Перечень ответственных конструкций",
                            "Коэффициент трудозатрат"],
                   'order_num': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]})

In [2089]:
cursor.execute(open("sql/37_2.sql", "r").read())
conn.commit()

In [2090]:
execute_values(conn, df, "general_data_sections")

execute_values() done


<h1>Пункты общих указаний марки</h1>

In [2091]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51],
                   'mark_id': [4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320],
                   'section_id': [1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 8, 9, 10, 10, 10, 11, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 15, 15, 15],
                   'text': ["- поручения от 28.02.2020 № АГЛ-35/0362",
                            "- письма от 05.03.2020 № УПиН-35/0431",
                            "- рекомендаций _, выполненного специалистами \"АО МАГНИТОГОРСКИЙ ГИПРОМЕЗ\"",
                            "- Федеральный закон от 30.12.2009 № 384-ФЗ \"Технический регламент о безопасности зданий и сооружений\"",
                            "- Федеральный закон от 22.07.2008 № 123-ФЗ \"Технический регламент о требованиях пожарной безопасности\"",
                            "- СП 16.13330.2017 \"Стальные конструкции. Актуализированная редакция СНиП II-23-81*\"",
                            "- СП 20.13330.2016 \"Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85*\"",
                            "- СП 28.13330.2017 \"Защита строительных конструкций от коррозии. Актуализированная редакция СНиП 2.03.11-85\"",
                            "- СП 53-102-2004 \"Общие правила проектирования стальных конструкций\"",
                            "- СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\"",
                            "- ГОСТ 23118-2012 \"Конструкции стальные строительные. Общие технические условия\"",
                            "- ГОСТ 27751-2014 \"Надежность строительных конструкций и оснований. Основные положения\"",
                            "- ГОСТ 9.402-2004 \"Единая система защиты от коррозии и старения (ЕСЗКС). Покрытия лакокрасочные. Подготовка металлических поверхностей к окрашиванию\"",
                            "- климатический район -- IB",
                            "- расчетная температура наружного воздуха наиболее холодных суток обеспеченностью 0,98 -- минус 43 {\degree}C",
                            "- нормативное значение веса снегового покрова на 1 м$^2$ горизонтальной поверхности земли для г. Магнитогорска (согласно СП 20.13330-2016, приложение K, табл. K1) -- 1,3 кПа (130 кг/м$^2$)",
                            "- нормативное значение ветрового давления для II района (согласно СП 20.13330-2016, п. 11.14, табл. 11.1) -- 0,30 кПа (30 кг/м$^2$)",
                            "- зона влажности -- сухая",
                            "- расчетная сейсмическая активность -- 5 баллов",
                            "# Чертежи марки \"КМ\" являются исходным материалом для разработки деталировочных чертежей марки \"КМД\"",
                            "# В данной рабочей документации разработано восстановление пролетных строений (усиление ферм пролетных строений, металлоконструкций по верхним и нижним поясам ферм) и восстановление антикоррозионного покрытия существующих элементов пролетных строений галереи конвейера № 706-1 УУК и КДИ аглоцеха ГОП ПАО \"ММК\"",
                            "# Все работы должны выполняться специализированной организацией, имеющей свидетельство СРО на данный вид деятельности и имеющие допуск по электробезопасности",
                            "# Работы производить по специально разработанному и согласованному в установленном порядке со всеми инспектирующими организациями и эксплуатацией цеха проекту производства работ (ППР) с соблюдением требований действующих нормативных документов и обязательным учетом фактического состояния существующих конструкций",
                            "# При выполнении работ принять меры по обеспечению целостности и сохранности существующих сооружений и коммуникаций, находящихся в зоне производства работ",
                            "# При производстве сварочных работ предусмотреть комплекс организационно-технических мероприятий по обеспечению пожарной безопасности",
                            "# На всех этапах ведения демонтажных и монтажных работ обеспечить устойчивость и неизменяемость положения существующих конструкций",
                            "# Перед началом работ необходимо выполнить очистку конструкций в зоне ремонта от мусора, грязи, технологической пыли",
                            "# Исполнитель несет ответственность за соответствие выполненных работ чертежам. Всякое отступление от чертежей при производстве работ должно быть согласовано с организацией-разработчиком данной рабочей документации",
                            "- коэффициент надежности по ответственности -- _",
                            "- степень агрессивного воздействия среды -- _",
                            "Марки сталей элементов конструкций приняты в зависимости от группы конструкций с учетом расчетной температуры и приведены в таблицах \"Ведомость элементов\", на схемах расположения элементов конструкций, на чертежах элементов и узлов конструкций и в спецификации металлопроката",
                            "Отступления от рабочих чертежей КМ не допускаются. В случае необходимости эти отступления должны быть согласованы с организацией-разработчиком рабочих чертежей марки \"КМ\"",
                            "# Изготовление металлоконструкций производить в соответствии с требованиями ГОСТ 23118-2012, СП 53-101-98 \"Изготовление и контроль качества стальных строительных конструкций\", указаниями в чертежах марки \"КМ\"",
                            "# Монтаж металлоконструкций производить в соответствии с требованиями СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\", СНИП 12-04-2002 \"Безопасность труда в строительстве. Часть 2. Строительное производство\" и указаниями в чертежах марки \"КМ\"",
                            "# Отклонения металлоконструкций при изготовлении и монтаже не должны превышать предельных значений, указанных в СП 53-101-98, СП 70.13330.2012 и в чертежах марки \"КМ\" данного основного комплекта",
                            "Минимальное усилие для расчетного прикрепления элементов -- 5 тс",
                            "- сварные",
                            "- на болтах нормальной точности по ГОСТ Р ИСО 4014-2013",
                            "# Материалы для сварки принимать по таблице Г.1 СП 16.13330.2017 в зависимости от марок стали свариваемых элементов",
                            "# Монтажные швы выполнять ручной сваркой ГОСТ 9467-75 \"Электроды покрытые металлические для ручной дуговой сварки конструкционных и теплоустойчивых сталей. Типы\":\nДля сталей C245 сварку производить электродами типа 342A",
                            "# Применение прерывистых швов не допускается",
                            "# Контроль качества монтажных сварных соединений должен осуществляться в соответствии с требованиями п. 10.4 СП 70.13330.2012",
                            "# Болты класса точности B (нормальной точности) M20 приняты по ГОСТ Р ИСО 4014-2013 \"Болты с шестигранной головкой. Классы точности A и B\", класса прочности 8.8 по ГОСТ ISO 898-1-2014 \"Механические свойства крепежных изделий из углеродистых и легированных сталей. Часть 1. Болты, винты и шпильки установленных классов прочности с крупным и мелким шагом резьбы\"",
                            "# Гайки тип 1 класса точности B приняты по ГОСТ ISO 4032-2014. При работе болтов на срез и растяжение класс прочности гаек принят в соответствии с классом прочности болтов: 5 -- при 5.6.\nШайбы приняты круглые по ГОСТ 11371-78 исполнения 1 класса точности A.\nДля болтов класса точности B диаметр отверстий в конструкциях, работающих на срез и смятие, принят d=dв+2.\nВ расчетных соединениях для болтов класса точности B разность номинальных диаметров отверстий и болтов не должна превышать 3 мм с учетом предельных отклонений диаметра отверстия и их овальности 0; +0,6; и 0; +1,0 по табл. 2 ГОСТ 23118-2012",
                            "# Антикоррозионная защита запроектирована в соответствии с требованиями СП 28.13330.2017 \"Защита строительных конструкций от коррозии\"",
                            "# Антикоррозионную защиту производить в соответствии с требованиями СП 72.13330.2016 \"Защита строительных конструкций и сооружений от коррозии. СНиП 3.04.03-85\" и ГОСТ 12.3.005-75 \"Система стандартов безопасности труда (ССБТ). Работы окрасочные. Общие требования безопасности\"",
                            "# Подготовка поверхностей перед окраской должна осуществляться в соответствии с требованиями ГОСТ 9.402-2004 путем удаления ржавчины и прокатной окалины с помощью ручного или механизированного абразивного инструмента и очистки жировых и прочих загрязнений с помощью волосяных кистей или ветоши, смоченных в уайт-спирите. Перед нанесением антикоррозионного покрытия поверхности конструкций обеспылить",
                            "# Защита металлоконструкций от коррозии осуществляется окраской лакокрасочными материалами группы III: эмалью XB-7141 по ТУ 2313-016-75351875-2005 в 4 слоя по 2 слоям грунтовки XC-010 по ТУ 6-21-7-89.\nОбщая толщина покрытия -- 160 мкм.\nСтепень очистки поверхности стальных конструкций от окислов перед окраской -- 2 по ГОСТ 9.402-2004n\nДопускается замена лакокрасочных материалов на сертифицированные материалы с аналогичными техническими характеристиками.\nЦвет окраски антикоррозионного покрытия принимать в соответствии с цветом окраски существующих конструкций",
                            "- подготовка поверхности металлоконструкций под окраску",
                            "- приемка грунтовочного слоя антикоррозионного покрытия",
                            "- визуальный контроль качества сварных швов"],
                   'order_num': [1, 2, 3, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 1, 1, 1, 2, 3, 1, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 1, 2, 3]})


In [2092]:
cursor.execute(open("sql/35_xx.sql", "r").read())
conn.commit()

In [2093]:
execute_values(conn, df, "mark_general_data_points")

execute_values() done


<h1>Пункты общих указаний пользователя (id seq fail)</h1>

In [2094]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51],
                   'user_id': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                   'section_id': [1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 8, 9, 10, 10, 10, 11, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 15, 15, 15],
                   'text': ["- поручения от 28.02.2020 № АГЛ-35/0362",
                            "- письма от 05.03.2020 № УПиН-35/0431",
                            "- рекомендаций _, выполненного специалистами \"АО МАГНИТОГОРСКИЙ ГИПРОМЕЗ\"",
                            "- Федеральный закон от 30.12.2009 № 384-ФЗ \"Технический регламент о безопасности зданий и сооружений\"",
                            "- Федеральный закон от 22.07.2008 № 123-ФЗ \"Технический регламент о требованиях пожарной безопасности\"",
                            "- СП 16.13330.2017 \"Стальные конструкции. Актуализированная редакция СНиП II-23-81*\"",
                            "- СП 20.13330.2016 \"Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85*\"",
                            "- СП 28.13330.2017 \"Защита строительных конструкций от коррозии. Актуализированная редакция СНиП 2.03.11-85\"",
                            "- СП 53-102-2004 \"Общие правила проектирования стальных конструкций\"",
                            "- СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\"",
                            "- ГОСТ 23118-2012 \"Конструкции стальные строительные. Общие технические условия\"",
                            "- ГОСТ 27751-2014 \"Надежность строительных конструкций и оснований. Основные положения\"",
                            "- ГОСТ 9.402-2004 \"Единая система защиты от коррозии и старения (ЕСЗКС). Покрытия лакокрасочные. Подготовка металлических поверхностей к окрашиванию\"",
                            "- климатический район -- IB",
                            "- расчетная температура наружного воздуха наиболее холодных суток обеспеченностью 0,98 -- минус 43 {\degree}C",
                            "- нормативное значение веса снегового покрова на 1 м$^2$ горизонтальной поверхности земли для г. Магнитогорска (согласно СП 20.13330-2016, приложение K, табл. K1) -- 1,3 кПа (130 кг/м$^2$)",
                            "- нормативное значение ветрового давления для II района (согласно СП 20.13330-2016, п. 11.14, табл. 11.1) -- 0,30 кПа (30 кг/м$^2$)",
                            "- зона влажности -- сухая",
                            "- расчетная сейсмическая активность -- 5 баллов",
                            "# Чертежи марки \"КМ\" являются исходным материалом для разработки деталировочных чертежей марки \"КМД\"",
                            "# В данной рабочей документации разработано восстановление пролетных строений (усиление ферм пролетных строений, металлоконструкций по верхним и нижним поясам ферм) и восстановление антикоррозионного покрытия существующих элементов пролетных строений галереи конвейера № 706-1 УУК и КДИ аглоцеха ГОП ПАО \"ММК\"",
                            "# Все работы должны выполняться специализированной организацией, имеющей свидетельство СРО на данный вид деятельности и имеющие допуск по электробезопасности",
                            "# Работы производить по специально разработанному и согласованному в установленном порядке со всеми инспектирующими организациями и эксплуатацией цеха проекту производства работ (ППР) с соблюдением требований действующих нормативных документов и обязательным учетом фактического состояния существующих конструкций",
                            "# При выполнении работ принять меры по обеспечению целостности и сохранности существующих сооружений и коммуникаций, находящихся в зоне производства работ",
                            "# При производстве сварочных работ предусмотреть комплекс организационно-технических мероприятий по обеспечению пожарной безопасности",
                            "# На всех этапах ведения демонтажных и монтажных работ обеспечить устойчивость и неизменяемость положения существующих конструкций",
                            "# Перед началом работ необходимо выполнить очистку конструкций в зоне ремонта от мусора, грязи, технологической пыли",
                            "# Исполнитель несет ответственность за соответствие выполненных работ чертежам. Всякое отступление от чертежей при производстве работ должно быть согласовано с организацией-разработчиком данной рабочей документации",
                            "- коэффициент надежности по ответственности -- _",
                            "- степень агрессивного воздействия среды -- _",
                            "Марки сталей элементов конструкций приняты в зависимости от группы конструкций с учетом расчетной температуры и приведены в таблицах \"Ведомость элементов\", на схемах расположения элементов конструкций, на чертежах элементов и узлов конструкций и в спецификации металлопроката",
                            "Отступления от рабочих чертежей КМ не допускаются. В случае необходимости эти отступления должны быть согласованы с организацией-разработчиком рабочих чертежей марки \"КМ\"",
                            "# Изготовление металлоконструкций производить в соответствии с требованиями ГОСТ 23118-2012, СП 53-101-98 \"Изготовление и контроль качества стальных строительных конструкций\", указаниями в чертежах марки \"КМ\"",
                            "# Монтаж металлоконструкций производить в соответствии с требованиями СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\", СНИП 12-04-2002 \"Безопасность труда в строительстве. Часть 2. Строительное производство\" и указаниями в чертежах марки \"КМ\"",
                            "# Отклонения металлоконструкций при изготовлении и монтаже не должны превышать предельных значений, указанных в СП 53-101-98, СП 70.13330.2012 и в чертежах марки \"КМ\" данного основного комплекта",
                            "Минимальное усилие для расчетного прикрепления элементов -- 5 тс",
                            "- сварные",
                            "- на болтах нормальной точности по ГОСТ Р ИСО 4014-2013",
                            "# Материалы для сварки принимать по таблице Г.1 СП 16.13330.2017 в зависимости от марок стали свариваемых элементов",
                            "# Монтажные швы выполнять ручной сваркой ГОСТ 9467-75 \"Электроды покрытые металлические для ручной дуговой сварки конструкционных и теплоустойчивых сталей. Типы\":\nДля сталей C245 сварку производить электродами типа 342A",
                            "# Применение прерывистых швов не допускается",
                            "# Контроль качества монтажных сварных соединений должен осуществляться в соответствии с требованиями п. 10.4 СП 70.13330.2012",
                            "# Болты класса точности B (нормальной точности) M20 приняты по ГОСТ Р ИСО 4014-2013 \"Болты с шестигранной головкой. Классы точности A и B\", класса прочности 8.8 по ГОСТ ISO 898-1-2014 \"Механические свойства крепежных изделий из углеродистых и легированных сталей. Часть 1. Болты, винты и шпильки установленных классов прочности с крупным и мелким шагом резьбы\"",
                            "# Гайки тип 1 класса точности B приняты по ГОСТ ISO 4032-2014. При работе болтов на срез и растяжение класс прочности гаек принят в соответствии с классом прочности болтов: 5 -- при 5.6.\nШайбы приняты круглые по ГОСТ 11371-78 исполнения 1 класса точности A.\nДля болтов класса точности B диаметр отверстий в конструкциях, работающих на срез и смятие, принят d=dв+2.\nВ расчетных соединениях для болтов класса точности B разность номинальных диаметров отверстий и болтов не должна превышать 3 мм с учетом предельных отклонений диаметра отверстия и их овальности 0; +0,6; и 0; +1,0 по табл. 2 ГОСТ 23118-2012",
                            "# Антикоррозионная защита запроектирована в соответствии с требованиями СП 28.13330.2017 \"Защита строительных конструкций от коррозии\"",
                            "# Антикоррозионную защиту производить в соответствии с требованиями СП 72.13330.2016 \"Защита строительных конструкций и сооружений от коррозии. СНиП 3.04.03-85\" и ГОСТ 12.3.005-75 \"Система стандартов безопасности труда (ССБТ). Работы окрасочные. Общие требования безопасности\"",
                            "# Подготовка поверхностей перед окраской должна осуществляться в соответствии с требованиями ГОСТ 9.402-2004 путем удаления ржавчины и прокатной окалины с помощью ручного или механизированного абразивного инструмента и очистки жировых и прочих загрязнений с помощью волосяных кистей или ветоши, смоченных в уайт-спирите. Перед нанесением антикоррозионного покрытия поверхности конструкций обеспылить",
                            "# Защита металлоконструкций от коррозии осуществляется окраской лакокрасочными материалами группы III: эмалью XB-7141 по ТУ 2313-016-75351875-2005 в 4 слоя по 2 слоям грунтовки XC-010 по ТУ 6-21-7-89.\nОбщая толщина покрытия -- 160 мкм.\nСтепень очистки поверхности стальных конструкций от окислов перед окраской -- 2 по ГОСТ 9.402-2004n\nДопускается замена лакокрасочных материалов на сертифицированные материалы с аналогичными техническими характеристиками.\nЦвет окраски антикоррозионного покрытия принимать в соответствии с цветом окраски существующих конструкций",
                            "- подготовка поверхности металлоконструкций под окраску",
                            "- приемка грунтовочного слоя антикоррозионного покрытия",
                            "- визуальный контроль качества сварных швов"],
                   'order_num': [1, 2, 3, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 1, 1, 1, 2, 3, 1, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 1, 2, 3]})

In [2095]:
cursor.execute(open("sql/35_2.sql", "r").read())
conn.commit()

Will fail first time. Need to create users.

In [2097]:
execute_values(conn, df, "general_data_points")

execute_values() done
